In [35]:
import pandas as pd
import json

In [70]:
import pandas as pd

# Load CSV
df = pd.read_csv("y4csv.csv")

valid_days = {"M", "T", "W", "Th", "F", "Sa", "Su"}

def expand_row(row):
    multiplier = row.get("multiplier")
    
    if pd.isna(multiplier) or str(multiplier).strip() == "":
        return row.to_frame().T  # Return a one-row DataFrame
    
    days = [day.strip() for day in str(multiplier).split(",") if day.strip() in valid_days]
    
    if not days:
        return row.to_frame().T

    # Create a DataFrame by repeating the row for each day
    expanded_df = pd.DataFrame([row] * len(days)).copy()
    expanded_df["templates"] = [f"{row['templates']}_{day}" for day in days]
    expanded_df["subtasks"] = [f"{row['subtasks']}_{day.lower()}" for day in days]
    
    return expanded_df

expanded_dfs = df.apply(expand_row, axis=1)  # this returns a Series of DataFrames
expanded_df = pd.concat(expanded_dfs.tolist(), ignore_index=True)
print(expanded_df['templates'])

0               HouseworkHelp
1           PersonAStartBlock
2             PersonAEndBlock
3           PersonBStartBlock
4             PersonBEndBlock
5           DentalAppointment
6             CommunityDinner
7             CommunityDinner
8             CommunityDinner
9             CommunityDinner
10         BuyCakeIngredients
11                PrepareFood
12            ExerciseClass_M
13            ExerciseClass_W
14           ExerciseClass_Th
15              LineDancing_W
16              LineDancing_F
17                   ALunch_M
18                   ALunch_T
19                   ALunch_W
20                  ALunch_Th
21                     Sewing
22                PCPACheckup
23    PCPACPrescriptionPickup
24               CallDaughter
25                   BLunch_M
26                   BLunch_T
27                   BLunch_W
28                  BLunch_Th
29                   BLunch_F
30                Volunteer_M
31                Volunteer_T
32                Volunteer_W
33        

/var/folders/j9/bdbyx62j20z5s3dgn8p4fl740000gn/T/ipykernel_54010/4293562303.py:27: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  expanded_df = pd.concat(expanded_dfs.tolist(), ignore_index=True)


In [71]:
templates = expanded_df
templates["requiredCapabilities"] = templates["requiredCapabilities"].astype(str).str.replace(" ", "")
all_capabilities = templates["requiredCapabilities"].str.split(",")
unique_capabilities = sorted(set(cap for sublist in all_capabilities for cap in sublist))
capability_to_id = {cap: str(i + 1) for i, cap in enumerate(unique_capabilities)}
templates["requiredCapabilityIds"] = all_capabilities.apply(
    lambda caps: ",".join(capability_to_id[cap] for cap in caps)
)
templates["start-location"] = templates["start-location"].fillna("@start-location")
templates["end-location"] = templates["end-location"].fillna("@end-location")
templates["id"] = range(1, len(templates) + 1)
unique_subtasks = {subtask: i + 1 for i, subtask in enumerate(sorted(templates["subtasks"].unique()))}
templates["subtask_id"] = templates["subtasks"].map(unique_subtasks)
templates["requiredCapabilities"] = templates["requiredCapabilities"].astype(str).str.replace(" ", "").str.split(",")
templates["requiredCapabilityIds"] = templates["requiredCapabilityIds"].astype(str).str.split(",")
templates['templates']


0               HouseworkHelp
1           PersonAStartBlock
2             PersonAEndBlock
3           PersonBStartBlock
4             PersonBEndBlock
5           DentalAppointment
6             CommunityDinner
7             CommunityDinner
8             CommunityDinner
9             CommunityDinner
10         BuyCakeIngredients
11                PrepareFood
12            ExerciseClass_M
13            ExerciseClass_W
14           ExerciseClass_Th
15              LineDancing_W
16              LineDancing_F
17                   ALunch_M
18                   ALunch_T
19                   ALunch_W
20                  ALunch_Th
21                     Sewing
22                PCPACheckup
23    PCPACPrescriptionPickup
24               CallDaughter
25                   BLunch_M
26                   BLunch_T
27                   BLunch_W
28                  BLunch_Th
29                   BLunch_F
30                Volunteer_M
31                Volunteer_T
32                Volunteer_W
33        

In [72]:
templates_list = []

for template_name, group in templates.groupby("templates"):
    template_id = group["id"].iloc[0]
    
    subtasks = []
    for _, row in group.iterrows():
        subtask = {
            "taskName": row["subtasks"],
            "id": int(row["subtask_id"]),
            "type": "executable",
            "requiredCapabilities": row["requiredCapabilities"],
            "requiredCapabilityIds": list(map(int, row["requiredCapabilityIds"])),
            "duration": int(row["duration"]),
            "start-location": row["start-location"],
            "end-location": row["end-location"],
            "explicit_transport_task": bool(row["explicit_transport_task"])
        }
        subtasks.append(subtask)
    
    template_entry = {
        "name": template_name,
        "id": int(template_id),
        "type": "meets",
        "requiredCapabilities": [],
        "subtasks": subtasks
    }
    templates_list.append(template_entry)

# Final dictionary
result_dict = {"templates": templates_list}
# Optional: save to JSON
with open("templates_output.json", "w") as f:
    json.dump(result_dict, f, indent=4)

In [74]:
unique_df = (
    templates.sort_values("templates")             # optional: deterministic order
      .drop_duplicates(subset="templates", keep="first")
)
orders = []
for _, row in unique_df.iterrows():
    orders.append({
        "name":              row["templates"],
        "quantity":          1,
        "earlieststartdate": "",            # left blank per your spec
        "duedate":           "",            # left blank per your spec
        "start-location":    row.get("start-location", "") or "",
        "end-location":      row.get("end-location", "")   or "",
        "tasks":             [row["templates"]],
        "optional":          False
    })

orders_dict = {"orders": orders}

with open("orders.json", "w") as f:
    json.dump(orders_dict, f, indent=4)